In [29]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from ast import literal_eval
import reverse_geocode



In [2]:
ruta_bussiness='Datasets/Yelp/business.pkl'

In [3]:
df_business = pd.read_pickle(ruta_bussiness)

In [4]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [5]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos las columnas que en este caso estan repetidas pero antes verificamos que los datos sean repetidos

In [6]:
# Obtener los nombres de las columnas como una lista
column_names = df_business.columns.tolist()

# Encontrar el índice de la segunda columna 'business_id'
index = [i for i, name in enumerate(column_names) if name == 'business_id'][1]

# Cambiar el nombre de la segunda columna 'business_id'
column_names[index] = 'bussiness 2'

# Asignar los nuevos nombres de las columnas al DataFrame
df_business.columns = column_names

In [7]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'bussiness 2', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [8]:
#Verificar que la segunda columna de business no es la misma que la otra
equal_values = (df_business['business_id'] == df_business['bussiness 2']).sum()

In [9]:
equal_values

5

Una vez verificado que las claves de los negocios son repetidas procedemos a eliminar dichas columnas.

In [10]:
df_business=df_business.iloc[:,:14]

In [11]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


Ahora procedemos a eliminar las columnas que no nos sirven para este proyecto.

In [12]:
df_business.drop(columns=['attributes'], inplace=True)

Si bien luego se realizara un análisis de la cantidad de nulos se eliminan las filas donde no tenemos datos acerca de la categoría.

In [13]:
#Eliminamos las filas de la columna 'categories' donde hay nulos
df_business.dropna(subset=['categories'], inplace=True)

In [14]:
df_business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
categories          0
hours           23120
dtype: int64

In [15]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,1,"Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,1,"Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,1,"Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,1,"Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


Comenzamos haciendo el filtro por resaurantes para luego proceder con los estados.

In [16]:
df=df_business.copy()

#Definimos las palabras claves
palabras_claves = ['restaurant', 'food','steakhouse','mexican','pizzeria','american','asian']

In [17]:
#Realizamos una función para convertir un string en una lista de palabras
def string_to_list(x):
    return [item.strip() for item in x.split(',')]

In [18]:
def filter_list_by_keywords(lst, palabras_claves):
    return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]


In [19]:
#Convertimos la columna 'categories' en una lista
df['categories'] = df['categories'].apply(string_to_list)

In [20]:
df['categories']

0         [Doctors, Traditional Chinese Medicine, Naturo...
1         [Shipping Centers, Local Services, Notaries, M...
2         [Department Stores, Shopping, Fashion, Home & ...
3         [Restaurants, Food, Bubble Tea, Coffee & Tea, ...
4                               [Brewpubs, Breweries, Food]
                                ...                        
150341                         [Nail Salons, Beauty & Spas]
150342    [Pets, Nurseries & Gardening, Pet Stores, Hobb...
150343    [Shopping, Jewelry, Piercing, Toy Stores, Beau...
150344    [Fitness/Exercise Equipment, Eyewear & Opticia...
150345    [Beauty & Spas, Permanent Makeup, Piercing, Ta...
Name: categories, Length: 150243, dtype: object

In [21]:
df['categories'] = df['categories'].apply(lambda x: filter_list_by_keywords(x, palabras_claves))

In [22]:
#Realizamos una funcion para extraer las palabras claves que no tengan que ver con el rubro
def filter_list_by_keywords(lst, palabras_claves):
    return [word for word in lst if any(palabra.lower() in word.lower() for palabra in palabras_claves)]

In [23]:
df['categories'].value_counts()

categories
[]                                                                                                         85614
[Restaurants]                                                                                              17032
[Food]                                                                                                      9610
[Food, Restaurants]                                                                                         3431
[Restaurants, Food]                                                                                         3267
                                                                                                           ...  
[American (New), Restaurants, Fast Food, Food, Food Delivery Services]                                         1
[American (Traditional), Food, Specialty Food, American (New), Comfort Food, Asian Fusion, Restaurants]        1
[Fast Food, Latin American, Restaurants]                                             

In [24]:
df = df[df['categories'].apply(lambda x: len(x) > 0)]

In [25]:
df.reset_index(drop=True, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64629 entries, 0 to 64628
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   64629 non-null  object
 1   name          64629 non-null  object
 2   address       64629 non-null  object
 3   city          64629 non-null  object
 4   state         64629 non-null  object
 5   postal_code   64629 non-null  object
 6   latitude      64629 non-null  object
 7   longitude     64629 non-null  object
 8   stars         64629 non-null  object
 9   review_count  64629 non-null  object
 10  is_open       64629 non-null  object
 11  categories    64629 non-null  object
 12  hours         55293 non-null  object
dtypes: object(13)
memory usage: 6.4+ MB


In [27]:
#Limpiamos los strings 
df['name'] = df['name'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['address'] = df['address'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['city'] = df['city'].apply(lambda x: x.strip() if isinstance(x, str) else x)
df['state'] = df['state'].apply(lambda x: x.strip() if isinstance(x, str) else x)


Vamos a verificar la ciudad y el estado

In [30]:
#Creamos una funcion para verificar la ciudad y el estado
def get_location(row):
    coordinates = [(row['latitude'], row['longitude'])]
    location = reverse_geocode.search(coordinates)[0]
    if location['country_code'] == 'US':
        return pd.Series([location['city'], location['state']])
    else:
        return pd.Series([None, None])

df[['city', 'state']] = df.apply(get_location, axis=1)

In [31]:
df['state'].value_counts()

state
Pennsylvania    15297
Florida         10864
Tennessee        5295
Indiana          5139
Missouri         4972
Louisiana        4520
New Jersey       4261
Arizona          3378
Nevada           2241
Idaho            1685
California       1595
Illinois         1276
Delaware         1143
Name: count, dtype: int64

In [32]:
#Cambiamos los nombres de los estados por sus abreviaturas de los estados que nos interesan
df['state'] = df['state'].map({'Florida':'FL','Texas':'TX','California':'CA','New York':'NY','Pennsylvania':'PA'})

In [33]:
#Filtramos los restaurantes por estado
lista_estados=['FL','TX','CA','NY','PA']
df = df[df['state'].isin(lista_estados)]

In [34]:
df.isnull().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
categories         0
hours           4007
dtype: int64

In [220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27756 entries, 0 to 64627
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   27756 non-null  object
 1   name          27756 non-null  object
 2   address       27756 non-null  object
 3   city          27756 non-null  object
 4   state         27756 non-null  object
 5   postal_code   27756 non-null  object
 6   latitude      27756 non-null  object
 7   longitude     27756 non-null  object
 8   stars         27756 non-null  object
 9   review_count  27756 non-null  object
 10  is_open       27756 non-null  object
 11  categories    27756 non-null  object
 12  hours         23749 non-null  object
dtypes: object(13)
memory usage: 3.0+ MB


In [221]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Center City,PA,19107,39.955505,-75.155564,4.0,80,1,"[Restaurants, Food]","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Woxall,PA,18054,40.338183,-75.471659,4.5,13,1,[Food],"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa,FL,33602,27.955269,-82.45632,4.0,10,1,"[Food, Restaurants, Food Trucks]","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
7,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"[Food, Restaurants]","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
8,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Center City,PA,19106,39.953949,-75.143226,4.0,245,1,[Restaurants],"{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."


In [35]:
#Se cambian los tipos de datos de latitude, logitude,stars,review_count
df = df.astype({
    'latitude': 'float64',
    'longitude': 'float64',
    'stars': 'float32',
    'review_count': 'int64'
})



In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27756 entries, 0 to 64627
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   27756 non-null  object 
 1   name          27756 non-null  object 
 2   address       27756 non-null  object 
 3   city          27756 non-null  object 
 4   state         27756 non-null  object 
 5   postal_code   27756 non-null  object 
 6   latitude      27756 non-null  float64
 7   longitude     27756 non-null  float64
 8   stars         27756 non-null  float32
 9   review_count  27756 non-null  int64  
 10  is_open       27756 non-null  object 
 11  categories    27756 non-null  object 
 12  hours         23749 non-null  object 
dtypes: float32(1), float64(2), int64(1), object(9)
memory usage: 2.9+ MB


In [37]:
df['state'].value_counts()

state
PA    15297
FL    10864
CA     1595
Name: count, dtype: int64

In [38]:
#Borramos la columna hours que finalmente no se usará
df.drop(columns=['hours'], inplace=True)

In [39]:
df.to_parquet('Datos limpios/restaurantes_yelps.parquet', index=False)